O curso de base para o estudo eh https://iros2015.informatik.uni-hamburg.de/index.html

Sera utilizado nesse estudo o CoppeliaSim e sua API remota ZeroMQ de forma a controlar o robo presente na cena de simulacao por meio de um script Python.

Para instalar a biblioteca que realiza a interacao usando ZeroMQ:

```
pip3 install coppeliasim_zmqremoteapi_client
```

Robo utilizado http://www.youbot-store.com/developers/documentation.

A seguir eh realizada a conexao entre o script e o simulador, que deve estar aberto no momento de execucao.

In [1]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import numpy as np
import math

print("Trying to connect")
client = RemoteAPIClient()
sim = client.require('sim')
print("Connected!")

Trying to connect
Connected!


A funcao a seguir sera aproveitada por outra rotina e tem como objetivo definir a posicao de cada junta do manipulador.

In [2]:
def arm_configuration_callback(config, vel, accel, handles):
    for i, joint_value in enumerate(config):
        if sim.isDynamicallyEnabled(handles[i]):
            sim.setJointTargetPosition(handles[i], joint_value)
        else:
            sim.setJointPosition(handles[i], joint_value)

Foi criada uma classe para utilizar o robo, esta classe tem 3 funcoes responsaveis por mover a base do robo, definir uma configuracao espacial para o manipulador, e definir uma configuracao para a garra.

Na funcao _move_ se pode ver implementada a cinematica do robo movel omnidirecional de 4 rodas "mecanum", como descrito no Modern Robotics 13.2.

![omni_kinematics](docs/omni_kinematics.png)

![omni_model](docs/omni_model.png)

Em `gripper_configuration` eh enviada uma lista de angulos de junta que serao interpolados com a posicao atual e a trajetoria gerada sera executada.

A junta 1 configura o deslocamento da garra inteira para a direcao do primeiro dedo e vai de 0 a 0.025. Default: 0.025.
A junta 2 configura a velocidade somente do segundo dedo, com a posicao indo de -0.05 a 0.0 e a velocidade negativa abrir a garra. Default: -0.05.

In [3]:
class YouBot:
    def __init__(self):
        self.l = 2.28
        self.w = 1.58
        self.wheels = {}
        self.wheels["front_left"] = sim.getObject('./rollingJoint_fl')
        self.wheels["rear_left"] = sim.getObject('./rollingJoint_rl')
        self.wheels["rear_right"] = sim.getObject('./rollingJoint_rr')
        self.wheels["front_right"] = sim.getObject('./rollingJoint_fr')

        if (self.wheels["front_left"] == -1): # TODO: completar o teste
            print("error")


        self.armJoints = []
        for i in range(5):
            self.armJoints.append(sim.getObject('./youBotArmJoint' + str(i)))

        self.gripperJoints = []
        for i in range(1, 3):
            self.gripperJoints.append(sim.getObject('./youBotGripperJoint' + str(i)))

        for handler in list(self.wheels.values()) + self.armJoints + self.gripperJoints:
            if handler == -1:
                print("handler failed")
        
        positions = ["NW", "BackE", "BackW", "FrontE", "FrontW", "SE", "SW"]
        self.ultrasonics = {}
        for position in positions:
            self.ultrasonics[position] = sim.getObject('./Ultrasonic' + position)


    def move(self, forward_vel, left_vel, rot_vel):
        u1 = - (self.l + self.w) * rot_vel + forward_vel - left_vel
        u2 = + (self.l + self.w) * rot_vel + forward_vel + left_vel
        u3 = + (self.l + self.w) * rot_vel + forward_vel - left_vel
        u4 = - (self.l + self.w) * rot_vel + forward_vel + left_vel

        sim.setJointTargetVelocity(self.wheels["front_left"], u1)
        sim.setJointTargetVelocity(self.wheels["front_right"], u2)
        sim.setJointTargetVelocity(self.wheels["rear_right"], u3)
        sim.setJointTargetVelocity(self.wheels["rear_left"], u4)

    def arm_configuration(self, joint_values, closed_gripper=False):

        vel=180
        accel=40
        jerk=80
       
    
        maxVel=[vel*math.pi/180,vel*math.pi/180,vel*math.pi/180,vel*math.pi/180,vel*math.pi/180,vel*math.pi/180]
        maxAccel=[accel*math.pi/180,accel*math.pi/180,accel*math.pi/180,accel*math.pi/180,accel*math.pi/180,accel*math.pi/180]
        maxJerk=[jerk*math.pi/180,jerk*math.pi/180,jerk*math.pi/180,jerk*math.pi/180,jerk*math.pi/180,jerk*math.pi/180]


        current_config=[]
        for joint in self.armJoints:
            current_config.append(sim.getJointPosition(joint))
        sim.moveToConfig(-1, current_config, [0]*5, [0]*5, maxVel,maxAccel,maxJerk,joint_values, [0]*5, arm_configuration_callback, self.armJoints)

        if closed_gripper:
            sim.setJointTargetVelocity(self.gripperJoints[1], 1.0)
    
    def gripper_configuration(self, gripper_values):
        if sim.isDynamicallyEnabled(self.gripperJoints[0]):
            sim.setJointTargetPosition(self.gripperJoints[0], gripper_values[0])
        else:
            sim.setJointPosition(self.gripperJoints[0], gripper_values[0])

        sim.setJointTargetVelocity(self.gripperJoints[1], gripper_values[1])

    def get_ultrasonics(self):
        ultrasonics_readings = {}
        for ultrasonic in self.ultrasonics:
            res, dist = sim.readProximitySensor(self.ultrasonics[ultrasonic])
            ultrasonics_readings[ultrasonic] = dist
        
        return ultrasonics_readings

Segue um exemplo do uso da API desenvolvida.

No exemplo se deseja demonstrar o movimento assim como remover o bloco presente na base com o manipulador.

In [5]:
def step_secs(secs, func):
    current = sim.getSimulationTime()
    while (t := sim.getSimulationTime()) - current < secs:
        print(f'Simulation time: {t:.2f} [s]')
        func()
        sim.step()

def main():
    sim.loadScene("/home/lucas/master/youbot/youbot.ttt")

    robot = YouBot()
    sim.setStepping(True)

    sim.startSimulation()

    # Se move em frente por 5 segundos
    func = lambda: robot.move(0.5, 0, 0)
    step_secs(5, func)

    # Se move a esquerda por 5 segundos
    func = lambda: robot.move(0, 0.5, 0)
    step_secs(5, func)

    # Rotaciona em torno do proprio centro por 5 segundos
    func = lambda: robot.move(0, 0, 0.5)
    step_secs(5, func)

    # Move o manipulador na direcao do bloco presente na base 
    func = lambda: robot.arm_configuration([0.0, 0.65, 0.915, 1.27, 0.0])
    step_secs(5, func)

    # Fecha a garra
    func = lambda: robot.gripper_configuration([0.025, 1.0])
    step_secs(3, func)

    # Move o manipulador para fora da base
    func = lambda: robot.arm_configuration([0.87, 0.61, 0.915, 1.27, 0.0])
    step_secs(2, func)

    func = lambda: robot.gripper_configuration([0.025, -0.04])
    step_secs(2, func)

    sim.stopSimulation()

if __name__ == "__main__":
    main()

Simulation time: 0.00 [s]
Simulation time: 0.05 [s]
Simulation time: 0.10 [s]
Simulation time: 0.15 [s]
Simulation time: 0.20 [s]
Simulation time: 0.25 [s]
Simulation time: 0.30 [s]
Simulation time: 0.35 [s]
Simulation time: 0.40 [s]
Simulation time: 0.45 [s]
Simulation time: 0.50 [s]
Simulation time: 0.55 [s]
Simulation time: 0.60 [s]
Simulation time: 0.65 [s]
Simulation time: 0.70 [s]
Simulation time: 0.75 [s]
Simulation time: 0.80 [s]
Simulation time: 0.85 [s]
Simulation time: 0.90 [s]
Simulation time: 0.95 [s]
Simulation time: 1.00 [s]
Simulation time: 1.05 [s]
Simulation time: 1.10 [s]
Simulation time: 1.15 [s]
Simulation time: 1.20 [s]
Simulation time: 1.25 [s]
Simulation time: 1.30 [s]
Simulation time: 1.35 [s]
Simulation time: 1.40 [s]
Simulation time: 1.45 [s]
Simulation time: 1.50 [s]
Simulation time: 1.55 [s]
Simulation time: 1.60 [s]
Simulation time: 1.65 [s]
Simulation time: 1.70 [s]
Simulation time: 1.75 [s]
Simulation time: 1.80 [s]
Simulation time: 1.85 [s]
Simulation t

[Lyapunov](https://www.researchgate.net/publication/374035623_Kinematic_Modeling_and_Stable_Control_Law_Designing_for_Four_Mecanum_Wheeled_Mobile_Robot_Platform_Based_on_Lyapunov_Stability_Criterion)

![error_matrix](docs/error_matrix.png)

In [ ]:
reference = np.array([1.0, 1.0, 0.5])
state = np.array([1.0, 1.0, 0.5])

error_transform = np.array([
    np.array([np.cos(state[2]), np.sin(state[2]),   0]),
    np.array([np.sin(state[2]), -np.cos(state[2]),  0]),
    np.array([0,                0,                  1]),
])

error = error_transform @ (reference - state)

![control](docs/control.png)

In [ ]:
# Reference velocity (Module must be constant)
v_r = 1
w_r = 0.2

# Parameters
k1 = 1.0
k2 = 1.0
k3 = 1.0

# Control law
v_c = v_r * np.cos(error[2]) + k2 * error[0]
w_c = w_r - (error[1] * v_r - k3 * np.sin(error[2])) / k1


Transform to fixed coordinate and apply forward kinematics

![fixed_frame](docs/fixed_frame.png)